In [59]:
import wikipedia
import pandas as pd
import spacy
import torch

from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16 = True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [2]:
df_en = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.en-train_nolabel.v1.jsonl', lines=True) #Cambiar para directorio local
df_en.head(5)

,lang,model_id,model_input,model_output_text,model_output_logits,model_output_tokens
0,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
1,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
2,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
3,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
4,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."


In [3]:
# Estas tres partes están bastante bien.
# El problema siento que radica en la siguiente celda donde recuperamos el resumen de cada página en vez de las secciones relevantes.
def noun_list(a, lang):
    """
    Filtra la pregunta y obtiene las PoST relevantes.
    
    a = list; Lista de preguntas del dataset
    lang = 'es' or 'en'; Idioma a trabajar
    """
    if lang == 'es':
        post_spacy = spacy.load("es_core_news_sm")
    else:
        post_spacy = spacy.load("en_core_web_sm")
    noun_list = []
    nums = ['0','1','2','3','4','5','6','7','8','9']

    for _ in a:
        doc = post_spacy(_)
        sub_noun = []
        for token in doc:
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "NUM":
                sub_noun.append(token.text)
            if token.pos_ == "ADJ" and token.text[0] in nums:
                sub_noun.append(token.text)
        noun_list.append(sub_noun)
    return noun_list

def keyword_por_preg(n_list):
    """
    Junta lista de PoST previo a pasarlo por el API de Wikipedia.
    
    n_list = list; Obtenida de la función noun_list().
    """
    keyword_list = []
    for i in n_list:
        keyword = ''
        for j in i:
            keyword = keyword + j + ' '
        keyword_list.append(keyword)
    return keyword_list

def get_wikipage(text, lang, page_total):
    """
    Regresa las n páginas de Wikipedia más relevantes al query

    text = str; Texto proveniente de la función keyword_por_preg()
    lang = 'es' or 'en'; Lenguaje necesario para wikipedia
    page_total = int; Cantidad de páginas a regresar
    """
    if lang == 'es':
        wikipedia.set_lang('es')
    if lang == 'en':
        wikipedia.set_lang('en')
    page_title = wikipedia.search(text, results = page_total)
    return page_title

In [42]:
def get_content(wiki_names):
    """
    Regresa la lista content con las páginas filtradas y segmentadas.

    wiki_names = list ; Lista de los nombres de páginas de Wikipedia. Extraído del dataset. 
    """
    p_content = []
    content = []
    
    for _ in wiki_names:
        text = ''
        try:
            page = wikipedia.WikipediaPage(_)
            textaux = page.content.replace('\n', '')
            textaux = textaux.replace('\t', '')
            text += '  ' + textaux
        except wikipedia.exceptions.DisambiguationError:
            print("Error")
        #text = page.content.replace('\n', '')
        #text = text.replace('\t', '')
        p_content.append(text)

    for _ in p_content:
        _ = _.split("===")
        texto = [i.replace('=', '') for i in _]
        aux = []
        for j in texto:
            if len(j) > 60:
                aux.append(j)
        content.append(aux)

    return content

def embeddings_t1(seg_txt):
    """
    Obtenemos los embeddings de cada elemento de seg_txt

    seg_txt = list ; Instancia única de content_list (content_list[i] for i in N)
    """
    len_list = [len(_) for _ in seg_txt]
    max_len = max(len_list)
    embedding_list = []
    for i in seg_txt:
        embs = model.encode(
            i,
            batch_size = 12,
            max_length = max_len,
        )["dense_vecs"]

        embeddings = [_ for _ in embs]
        embedding_list.append(embeddings)
    return embedding_list

def get_content_embs(content_list):
    """
    Genera la lista de content_embeddings a partir de la lista "content" correspondiente.

    content_list  = list ; lista obtenida 
    """
    content_embs = [embeddings_t1(_) for _ in content_list]
    return content_embs

In [5]:
def ds_procesamiento(dataset, lang):
    """
    Regresa un dataset bien formateado, acá chido para todo el procesamiento.
    
    dataset = pd.DataFrame ; El dataset mismo, así encuerado.
    lang = str ; 'es' or 'en' Para determinar el idioma.
    """
    a = keyword_por_preg(noun_list(dataset["model_input"], lang))
    dataset["output_filtrado"] = a

    a_set = list(set(a))
    dic = {}
    for _ in a_set:
        aux = get_wikipage(_, lang, 3)
        if not aux:
            print(f"Problema con: {_}")
        dic[f"{_}"] = aux

    aux1 = []
    for i in range(len(dataset["output_filtrado"])):
        if dataset["output_filtrado"][i] in dic:
            aux1.append(dic[dataset["output_filtrado"][i]])
    dataset["Wiki Asociado"] = aux1
    return dataset

In [30]:
# Preg_Emb
def similarity(t1, t2):
    return t1 @ t2

def q_emb(pregunta):
    """
    Genera embedding de una pregunta.

    pregunta = dataset["model_input"][i] ; Pregunta a vectorizar
    """
    preb_emb = model.encode(
        pregunta,
        batch_size=12,
        max_length = 512,
    )['dense_vecs']
    return preg_emb

In [23]:
def resumen_topn(texto, embeddings, pregunta, top_n):
    """
    texto = list ; lista obtenida de get_content()
    embeddings = list ; lista obtenida de get_content_embs()
    pregunta = BGE Embedding ; Correspondiente a una pregunta del dataset.
    top_n = int ; Cantidad de vectores usados para el resumen.
    """
    full = []
    resumen = ''
    for i in range(len(texto)):
        aux = list(zip(texto[i], embeddings[i]))
        for _ in aux:
            full.append(_)

    simi_list = [(similarity(pregunta, _[1]), _[0]) for _ in full]
    simi_list.sort(reverse=True)

    for i in simi_list[:top_n]:
        resumen += '---' + i[1]
    return resumen

In [6]:
%%time
aux_n = ds_procesamiento(df_en, 'en')
aux_n.head(3)

Problema con: Bischofsheim constitutent community Mainz 
CPU times: total: 2.16 s
Wall time: 20.3 s


,lang,model_id,model_input,model_output_text,model_output_logits,model_output_tokens,output_filtrado,Wiki Asociado
0,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...",arthropods,"[Arthropod, 2018 in arthropod paleontology, Ar..."
1,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...",arthropods,"[Arthropod, 2018 in arthropod paleontology, Ar..."
2,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...",arthropods,"[Arthropod, 2018 in arthropod paleontology, Ar..."


In [32]:
temp_wiki = aux_n["Wiki Asociado"][0]
temp_wiki

['Arthropod', '2018 in arthropod paleontology', 'Arthropod exoskeleton']

In [33]:
%%time
content = get_content(temp_wiki)
content_embs = get_content_embs(content)

CPU times: total: 2.48 s
Wall time: 3.83 s


In [34]:
preg = df_en["model_input"][0]
p_emb = q_emb(preg)

In [35]:
%%time
res1 = resumen_topn(content, content_embs, p_emb, 5)

CPU times: total: 0 ns
Wall time: 1e+03 µs


In [47]:
aux = df_en.iloc[0]
aux

lang                                                                  EN
model_id                            togethercomputer/Pythia-Chat-Base-7B
model_input                             Do all arthropods have antennae?
model_output_text       Yes, all insects and arachnids (including spi...
model_output_logits    [-2.57427001, 5.1865358353, 5.4173498154, 2.32...
model_output_tokens    [ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...
output_filtrado                                              arthropods 
Wiki Asociado          [Arthropod, 2018 in arthropod paleontology, Ar...
Name: 0, dtype: object

In [54]:
def emb_aux(dataset):
    """
    dataset = pd.DataFrame.iloc[i]
    """
    contenido = get_content(dataset["Wiki Asociado"])
    contenido_embs = get_content_embs(contenido)
    question = dataset["model_input"]
    question_emb = q_emb(question)
    return resumen_topn(contenido, contenido_embs, question_emb, 5)

In [60]:
%%time
resumen = []
for i in range(len(df_en)):
    try:
        resumen.append(emb_aux(df_en.iloc[i]))
    except ValueError:
        print(df_en.iloc[i]["model_input"])
        ola = "There's is no extra information. Answer with your own information."
        resumen.append(ola)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Error
What’s the real name of Edu Manga?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?
Error
What was the previous name of the Gillette Stadium?


AttributeError: module 'torch' has no attribute 'OutofMemoryError'

In [36]:
def emb_based_summary(dataset):
    """
    dataset = pd.DataFrame ; 
    """
    resumen_list = []
    for i in range(len(dataset["Wiki Asociado"])):
        contenido = get_content(dataset["Wiki Asociado"][i])
        contenido_embs = get_content_embs(contenido)
        question = dataset["model_input"][i]
        question_emb = q_emb(question)
        resumen = resumen_topn(contenido, contenido_embs, question_emb, 5)
        resumen_list.append(resumen)
    return resumen_list

In [43]:
%%time
ayuda = emb_based_summary(df_en)
print(len(ayuda), len(df_en["Wiki Asociado"]))

Error


ValueError: max() arg is an empty sequence